# Prompt versioning

<a target="_blank" href="https://colab.research.google.com/github/masci/banks/blob/main/cookbook/Prompt_Versioning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

A prompt is almost never set in stone. It can change over time as we find nuances in the language that improve 
performance, it can change as the model being used gets update, it almost certainly changes when the same prompt is 
used against different models.

In all these situations, being able to attach a version to a prompt can greatly help to keep things tidy, organized 
and ultimately save time. Let's see how to do this with Banks.

In [ ]:
!pip install banks

We'll store our templates in a local folder called `templates`.

In [ ]:
import os

os.mkdir("templates")

We now write two versions of the same prompt, optimized for different LLMs. The two prompts will share the same
`name` but will have different `version`s.

In [ ]:
from pathlib import Path

from banks import Prompt
from banks.registries import DirectoryPromptRegistry

# Tell the registry where prompt texts are stored
registry = DirectoryPromptRegistry(Path(".") / "templates")

# Write two versions of the same prompt, optimized for different LLMs
blog_prompt_gpt = Prompt("Write a 500-word blog post on {{ topic }}.\n\nBlog post:", name="blog_prompt", version="gpt-3.5-turbo")
# Llama usually benefits a lot from in-context learning, let's add examples
blog_prompt_llama3 = Prompt(
    "Write a blog post abot the topic {{ topic }}. Do not write more than 500 words"
    "Examples:"
    "{% for example in examples %}"
    "{{ example }}"
    "{% endfor %}"
    "\n\nBlog post:", name="blog_prompt", version="ollama_llama3.1:8b"
)

# Store the two prompts
registry.set(prompt=blog_prompt_gpt)
registry.set(prompt=blog_prompt_llama3)

When we need a prompt, we can now ask the registry for the version of choice.

In [ ]:
import os

from litellm import completion


## set ENV variables
os.environ["OPENAI_API_KEY"] = "your-api-key"


response = completion(
    model="gpt-3.5-turbo",
    messages=registry.get(name="blog_prompt", version="gpt-3.5-turbo").chat_messages(),
)